
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://moodle.df.senac.br/faculdade/pluginfile.php/1/theme_lambda/logo/1716924091/Logo-SENAC-PNG.png" style="width: 300px; height: 150px">
</div>


## Arquitetura Medalhão

A <a href='https://www.databricks.com/glossary/medallion-architecture' target='_blank'>arquitetura medalhão</a> é um padrão de design de dados usado para organizar logicamente os dados em um _lakehouse_, com o objetivo de melhorar de forma incremental e progressiva a estrutura e a qualidade dos dados à medida que fluem através de cada camada da arquitetura ( Bronze > Silver > Gold) . As arquiteturas Medalhão também são chamadas de arquiteturas <a href='https://www.linkedin.com/pulse/multi-hop-data-architecture-addiction-rick-van-der-lans-5xuke/' tagert='_blank'>_multi-hop_</a>.

![Medallion](https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png?v=1702318922)

### Criando tabelas para persistir nossos dados transformados

Com base na arquitetura medalhão, vamos evoluir nossos dados por três camadas. E para cada camada, vamos criar suas respectivas tabelas.

Para se criar uma tabela, o comando <a href='https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-table-using.html' target='_blank'>CREATE TABLE</a> pode ser usado. 


### Bronze
Na camada *bronze* os dados são carregados de forma _raw_, assim como estão na origem:

In [0]:
%sql
CREATE OR REPLACE TABLE exemplo_bronze (nome STRING, valor INT, tipo STRING) ;
INSERT INTO exemplo_bronze 
VALUES ("João", 1 , "t1|t2|t3"),
("Maria", 2, "t2|t5");

DESCRIBE extended exemplo_bronze;

SELECT * FROM exemplo_bronze;

nome,valor,tipo
João,1,t1|t2|t3
Maria,2,t2|t5


No comando acima, uma tabela foi criada no _schema default_. Navegue no catálogo e verifique a tabela.

Você pode inserir dados em uma tabela com base em uma consulta. Veja:


In [0]:
%sql
INSERT INTO exemplo_bronze
select * from VALUES ("José", 3, "t1|t2|t3|t7") , ("José", 7, "t3|t7");

select * from exemplo_bronze;

nome,valor,tipo
José,3,t1|t2|t3|t7
José,7,t3|t7
João,1,t1|t2|t3
Maria,2,t2|t5


### Silver
Na camada _silver_ realizamos algumas transformações, limpeza e tratamento nos dados, assim, cientistas de dados e analistas de dados já podem começar a trabalhá-los.

A camada _silver_ é alimentada pela camada _bronze_.

In [0]:
%sql
CREATE OR REPLACE TABLE exemplo_silver (nome STRING, valor INT, tipo STRING) ;

INSERT INTO exemplo_silver 
SELECT nome, valor, explode(split(tipo , '\\\|')) as tipo FROM exemplo_bronze;

SELECT * FROM exemplo_silver;

nome,valor,tipo
João,1,t1
João,1,t2
João,1,t3
Maria,2,t2
Maria,2,t5
José,3,t1
José,3,t2
José,3,t3
José,3,t7
José,7,t3


### Gold
Na camada _gold_ trabalhamos o dado para ser melhor apresentado e explorado pelo usuário final, podemos criar visões por produto, _star schema_, _snowflake_, visões agregadas, etc.

A camada _gold_ é alimentada pela camada _silver_.

In [0]:
%sql
CREATE OR REPLACE TABLE exemplo_gold (nome STRING, valor INT, tipo STRING) ;

INSERT INTO exemplo_gold 
SELECT nome, sum(valor), tipo FROM exemplo_silver group by nome, tipo;
--somando o valor dos dados em que os tipo sao iguais.
SELECT * FROM exemplo_gold;

nome,valor,tipo
Maria,2,t2
José,3,t1
João,1,t3
João,1,t2
Maria,2,t5
João,1,t1
José,10,t3
José,10,t7
José,3,t2


### Atividade

Com base na tabela `exemplo_gold`, elabore uma consulta que faça um _pivot_ por tipo, totalizando (_sum_) a coluna `valor`. Agrupe também pela coluna `nome`.

In [0]:

df = spark.sql("select * from exemplo_gold")
#df.show()
df.groupBy('nome').pivot('tipo').sum('valor').display()

nome,t1,t2,t3,t5,t7
José,3,3,10,null,10
João,1,1,1,null,null
Maria,null,2,null,2,null
